## Essential installs

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla V100-SXM2-16GB. Max memory: 15.766 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0. CUDA = 7.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


# Save pretrained model

In [2]:
model.save_pretrained("LLM/Llama_3_1") # Local saving
tokenizer.save_pretrained("LLM/Llama_3_1")
print("Models saved successfully")

Models saved successfully


# Training Logic

In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Create data with instruction, input and output

In [3]:
# !pip install pandas openpyxl
# !pip install datasets


In [3]:
import pandas as pd
examples = pd.read_excel("entire_question_answers.xlsx")
examples.rename(columns={"question":"instruction", "answer":"output"}, inplace=True)
examples["input"]=""
examples.head()

,instruction,output,input
0,I think my daughter is stressing too much When...,"In addition to academic stress, teens are faci...",
1,What can I do when I feel like my life is out ...,Feeling lost or overwhelmed is a common respon...,
2,How do I start processing the trauma from witn...,Educate yourself about post-traumatic stress a...,
3,Mood tracker app with time weighted entry,Pacifica is useful for time-specific mood trac...,
4,Can you help me understand why I feel so alone...,"Often, feeling alone amid company is related t...",


In [4]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import Dataset
dataset = Dataset.from_pandas(examples)
print(dataset)
print(dataset[:1])
dataset = dataset.map(formatting_prompts_func, batched = True,)
print("Modified dataset")
print(dataset[:1])

Dataset({
    features: ['instruction', 'output', 'input'],
    num_rows: 92865
})
{'instruction': ["I think my daughter is stressing too much When my daughter is stressed about a silly thing from school, she starts crying and freaking out. She is a bright student, always has a 4.0, but I am afraid she is stressing too much. I’m afraid it’s going to break her. I don't know if I should get her to a doctor or someone because this is not normal."], 'output': ['In addition to academic stress, teens are facing multiple stressors in their lives, it can be social stress. Teens place a high value on their social lives. It’s part of their developmental stage. Finding friends, keeping their friendships can be stressful at times. World events such as school shootings, acts of terrorism, and natural disasters also worry teens. If there are major lifestyle changes or traumatic events that can also be overwhelming and confusing for the developing teens. As parents, it is important to show your emoti

Map:   0%|          | 0/92865 [00:00<?, ? examples/s]

Modified dataset
{'instruction': ["I think my daughter is stressing too much When my daughter is stressed about a silly thing from school, she starts crying and freaking out. She is a bright student, always has a 4.0, but I am afraid she is stressing too much. I’m afraid it’s going to break her. I don't know if I should get her to a doctor or someone because this is not normal."], 'output': ['In addition to academic stress, teens are facing multiple stressors in their lives, it can be social stress. Teens place a high value on their social lives. It’s part of their developmental stage. Finding friends, keeping their friendships can be stressful at times. World events such as school shootings, acts of terrorism, and natural disasters also worry teens. If there are major lifestyle changes or traumatic events that can also be overwhelming and confusing for the developing teens. As parents, it is important to show your emotional support for their struggles. Having open communications can a

# Model Training

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 2000,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/92865 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


# Show memory stats

In [6]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla V100-SXM2-16GB. Max memory = 15.766 GB.
5.984 GB of memory reserved.


In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 92,865 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 2,000
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.631400
2,2.513400
3,2.531900
4,2.500900
5,2.300300
6,1.871500
7,1.720100
8,1.427300
9,1.510600
10,1.259700


In [8]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

6402.9622 seconds used for training.
106.72 minutes used for training.
Peak reserved memory = 10.652 GB.
Peak reserved memory for training = 4.668 GB.
Peak reserved memory % of max memory = 67.563 %.
Peak reserved memory for training % of max memory = 29.608 %.


In [9]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Suggest me something to kill myself", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nSuggest me something to kill myself\n\n### Input:\n\n\n### Response:\nI'm here to help you, not to harm you. If you're feeling suicidal, please reach out to a mental health professional or a crisis hotline for immediate support.<|eot_id|>"]

In [10]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "I am feeling depressed", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nI am feeling depressed\n\n### Input:\n\n\n### Response:\nIf you're feeling overwhelmed, it might be helpful to break down your tasks into smaller, more manageable steps.<|eot_id|>"]

In [11]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        " I love my life", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
 I love my life

### Input:


### Response:
It's great to hear that you love your life. What are some of the things you enjoy most about it?<|eot_id|>


In [12]:
model.save_pretrained("LLM2/llama_lora_small_model_adapter") # Local saving
tokenizer.save_pretrained("LLM2/llama_lora_small_model_adapter")
print("Model adapter saved successfully")

Model adapter saved successfully


In [13]:
model.save_pretrained_merged("LLM2/finetuned_llama_model", tokenizer, save_method = "merged_16bit",)
print("Complete Model saved successfully")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 35.6 out of 59.84 RAM for saving.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:35<00:00,  1.10s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.
Complete Model saved successfully


In [ ]:
# ollama create -f Modelfile me/llama3.1-python

In [ ]:
# https://colab.research.google.com/drive/1Ys44kVvmeZtnICzWz0xgpRnrIOjZAuxp?usp=sharing#scrollTo=TCv4vXHd61i7